In [1]:
import json
import vk_api

%matplotlib inline
from matplotlib import rc, cm
import matplotlib.pyplot as plt
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('../../vk_api_credentials.json', 'r') as json_file:
    client_credentials = json.load(json_file)

vk_session = vk_api.VkApi(
    client_credentials['email_or_phone_number'],
    client_credentials['password'],
)
vk_session.auth()
vk_client = vk_session.get_api()

In [3]:
# Это я.
user_id = 35201895

In [4]:
vk_friends = vk_client.friends.get(
    user_id=user_id,
    fields=['sex', 'city', 'education'],
    order='name'
)

In [5]:
vk_friends.keys()

dict_keys(['count', 'items'])

In [6]:
vk_friends['count']

311

In [47]:
vk_client.likes.getList(type='post', owner_id=-131489096, item_id=6682)['count']

108

In [23]:
random.randint(0, 1)

0

In [13]:
page_domain_name = 'dobriememes'
LIKES_THRESHOLD = 150
PHOTO_WIDTH_THRESHOLD = 600
POST_URL = "https://vk.com/{domain}?w=wall{owner_id}_{item_id}"


wall_posts = vk_client.wall.get(
    domain=page_domain_name,
    count=100,
)['items']
wall_posts[0]

{'id': 2215300,
 'from_id': -147286578,
 'owner_id': -147286578,
 'date': 1625502006,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'is_pinned': 1,
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1625502006,
    'id': 457602724,
    'owner_id': -147286578,
    'has_tags': False,
    'access_key': '1f68bea487a138b84a',
    'post_id': 2215300,
    'sizes': [{'height': 130,
      'url': 'https://sun1-47.userapi.com/impg/GlrrldKQm6CsvsuGdH678-C8jnTesxS_5RaMNw/XNh3gJCJOvo.jpg?size=117x130&quality=96&sign=a8bba5bc8581fac6445cef83d98b078f&c_uniq_tag=FqPdJDmynTh6BnNMx-cnSMWnoFeunXVR2uA3HFxXwVw&type=album',
      'type': 'm',
      'width': 117},
     {'height': 145,
      'url': 'https://sun1-47.userapi.com/impg/GlrrldKQm6CsvsuGdH678-C8jnTesxS_5RaMNw/XNh3gJCJOvo.jpg?size=130x145&quality=96&sign=ee495ee17c6d3b92e456133a1fec8006&c_uniq_tag=s7AFJmT204l1sFiTxAt6cmp_LwMbBI2wH6FMfgyQ9Ms&type=album',
      'type': 'o',
      'width': 130},
     {'height': 223,
 

In [15]:
page_domain_name = 'dobriememes'
LIKES_THRESHOLD = 150
PHOTO_WIDTH_THRESHOLD = 600
POST_URL = "https://vk.com/{domain}?w=wall{owner_id}_{item_id}"


wall_posts = vk_client.wall.get(
    domain=page_domain_name,
    count=100,
)['items']

photo_urls = []
photo_credits = []

for wall_post in wall_posts:
    if 'attachments' not in wall_post:
        continue
    likes_count = vk_client.likes.getList(
        type='post',
        owner_id=wall_post['owner_id'],
        item_id=wall_post['id'],
    )['count']
    photo = wall_post['attachments'][0].get('photo')
    if photo and likes_count > LIKES_THRESHOLD:
        for photo_item in photo['sizes']:
            if photo_item['width'] > PHOTO_WIDTH_THRESHOLD:
                photo_urls.append(photo_item['url'])
                photo_credits.append(
                    POST_URL.format(
                        domain=page_domain_name,
                        owner_id=wall_post['owner_id'],
                        item_id=wall_post['id'],
                    )
                )
                break

In [18]:
import random
import json
import vk_api

from typing import Dict, Tuple, List


class VkRandomMemeFinder:

    PHOTO_WIDTH_THRESHOLD = 600
    POST_URL = "https://vk.com/{domain}?w=wall{owner_id}_{item_id}"
    
    def __init__(
            self,
            vk_client,
            page_domain_name: str,
            likes_threshold: int,
    ):
        self._vk_client = vk_client
        self._page_domain_name = page_domain_name
        self._likes_threshold = likes_threshold
        self._collect_memes()
        
    @staticmethod
    def from_config(
            json_path: str,
            page_domain_name: str,
            likes_threshold: int,
    ):
        with open(json_path, 'r') as json_file:
            client_credentials = json.load(json_file)

        vk_session = vk_api.VkApi(
            client_credentials['email_or_phone_number'],
            client_credentials['password'],
        )
        vk_session.auth()
        vk_client = vk_session.get_api()
        return VkRandomMemeFinder(
            vk_client,
            page_domain_name,
            likes_threshold,
        )
    
    def __call__(self) -> Dict[str, str]:
        # Get random meme from a public page.
        index = random.randint(0, self._memes_count - 1)
        return {
            'meme_url': self._photo_urls[index],
            'meme_credit': self._photo_credits[index],
        }
        
    
    def _collect_memes(self) -> List[Tuple[str, str]]:
        wall_posts = self._vk_client.wall.get(
            domain=self._page_domain_name,
            count=100,
        )['items']

        self._photo_urls = []
        self._photo_credits = []

        for wall_post in wall_posts:
            if 'attachments' not in wall_post:
                continue
            likes_count = self._vk_client.likes.getList(
                type='post',
                owner_id=wall_post['owner_id'],
                item_id=wall_post['id'],
            )['count']
            photo = wall_post['attachments'][0].get('photo')
            if photo and likes_count > self._likes_threshold:
                for photo_item in photo['sizes']:
                    if photo_item['width'] > self.PHOTO_WIDTH_THRESHOLD:
                        self._photo_urls.append(photo_item['url'])
                        self._photo_credits.append(
                            self.POST_URL.format(
                                domain=self._page_domain_name,
                                owner_id=wall_post['owner_id'],
                                item_id=wall_post['id'],
                            )
                        )
                        break
        self._memes_count = len(self._photo_urls)
        return zip(self._photo_urls, self._photo_credits)

In [19]:
#ML_MEMES_DOMAIN_NAME = 'weirdreparametrizationtrick'
DOBRIE_MEMES_DOMAIN_NAME = 'dobriememes'

vk_random_meme_finder = VkRandomMemeFinder.from_config(
    json_path='../../vk_api_credentials.json',
    page_domain_name=DOBRIE_MEMES_DOMAIN_NAME,
    likes_threshold=3000,
)

In [15]:
vk_random_meme_finder()

{'meme_url': 'https://sun9-80.userapi.com/impg/2Uw9vaJNGMZsRrufCjGk4RJTFkGrI4eg4zLVtw/vSffQvapMqw.jpg?size=604x328&quality=96&sign=0d126fb1a730502a3d2ac509dd74ebe1&c_uniq_tag=Z5G7gNqmJlOhKbiuZ1SSTCJHOe5IKHzlxDjTpxAQQx8&type=album',
 'meme_credit': 'https://vk.com/weirdreparametrizationtrick?w=wall-131489096_6763'}